In [ ]:
# Todo.
# 1. List all my heroes with floor price and highest bid price - lets do this now. JUST NEED TO ADD BID PRICE - DONE
# 2. Value portfolio by last trade - Should be possible with the above - DONE
# 3. Note differentials with bids and listings.
# 4. Chart performance and prices
# 5. Work out average tournament score - DONE
# 7. Work out when scores gonna drop.

# Example usage
PLAYER_ID = "0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a"


In [16]:
import os
import re
import json
import time
import random
import glob
import sys
import requests
import pandas as pd
import pickle
from datetime import datetime, timedelta
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent

# Global Variables
PLAYER_ID = "0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a"
URL_GRAPHQL = "https://fantasy-top.hasura.app/v1/graphql"
URL_REST = "https://fantasy.top/api/bids/get-bid-orders"
COOKIES_FILE = 'cookies.pkl'
DATA_FOLDER = "data"

# Utility Functions

def save_cookies(driver, filepath=COOKIES_FILE):
    with open(filepath, 'wb') as file:
        pickle.dump(driver.get_cookies(), file)

def load_cookies(driver, filepath=COOKIES_FILE):
    try:
        with open(filepath, 'rb') as file:
            cookies = pickle.load(file)
            for cookie in cookies:
                driver.add_cookie(cookie)
    except FileNotFoundError:
        print("Cookies file not found. Proceeding without loading cookies.")

def convert_to_eth(value):
    try:
        if pd.isna(value):
            return None
        return int(float(value)) / 1e18
    except (ValueError, TypeError) as e:
        print(f"Error converting value {value}: {e}")
        return None

def save_df_as_csv(df, filename, folder=DATA_FOLDER):
    timestamp = datetime.now().strftime('%y%m%d_%H%M')
    filename_with_timestamp = f"{filename}_{timestamp}.csv"
    if not os.path.exists(folder):
        os.makedirs(folder)
    full_path = os.path.join(folder, filename_with_timestamp)
    df.to_csv(full_path, index=False)
    print(f"DataFrame saved as {full_path}")


def print_runtime(func, *args, **kwargs):
    print(f'Calling {func.__name__}')
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    runtime = end_time - start_time
    print(f'{func.__name__} took {runtime:.4f} seconds to execute')
    return result

# WebDriver and Login

def get_random_user_agent():
    ua = UserAgent(platforms='pc')
    return ua.random

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument(f"user-agent={get_random_user_agent()}")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--disable-notifications")
    options.add_argument("--disable-popup-blocking")
    options.add_argument("--auto-open-devtools-for-tabs")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--headless")
    options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
    driver = webdriver.Chrome(options=options)
    return driver

def login_to_fantasy(driver, username, password):
    driver.get("https://www.fantasy.top/home")
    wait = WebDriverWait(driver, 10)
    button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[style*='background: linear-gradient'][class*='rounded-md']")))
    button.click()
    modal_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Twitter')]")))
    modal_button.click()
    time.sleep(2)
    actions = ActionChains(driver)
    actions.send_keys(Keys.ESCAPE).perform()
    username_input = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='text'][autocomplete='username']")))
    username_input.send_keys(username)
    next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Next']/ancestor::button")))
    next_button.click()
    password_input = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password'][type='password']")))
    password_input.send_keys(password)
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='LoginForm_Login_Button']")))
    login_button.click()
    close_popup_if_appears(driver)
    authorize_if_appears(driver)
    accept_terms_if_appears(driver)
    driver.get("https://www.fantasy.top/home")
    print("Logged In Successfully")

def close_popup_if_appears(driver):
    try:
        close_popup_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='xMigrationBottomBar']"))
        )
        close_popup_button.click()
    except TimeoutException:
        print("Close popup button did not appear.")

def authorize_if_appears(driver):
    try:
        authorize_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='OAuth_Consent_Button']"))
        )
        authorize_button.click()
    except TimeoutException:
        print("Authorize app button did not appear.")

def accept_terms_if_appears(driver):
    try:
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.sc-fqkvVR.sc-iGgWBj.dQeymh.httOiR"))
        )
        accept_button.click()
    except TimeoutException:
        print("Accept button did not appear.")

def login():
    username = "MissPecuIiar"
    password = "Dublin84"
    driver = setup_driver()
    driver.get("https://www.fantasy.top/home")
    load_cookies(driver)
    driver.refresh()
    login_to_fantasy(driver, username, password)
    token = driver.execute_script("return localStorage.getItem('jwtToken');")
    save_cookies(driver)
    time.sleep(2)
    actions = ActionChains(driver)
    actions.send_keys(Keys.ESCAPE).perform()
    return driver, token

# Data Download Functions

def download_listings(driver):
    driver.get('https://fantasy.top/marketplace')
    time.sleep(5)
    actions = ActionChains(driver)
    actions.send_keys(Keys.ESCAPE).perform()
    try:
        main_element = driver.find_element(By.TAG_NAME, "main")
        main_element.click()
    except ElementClickInterceptedException:
        actions.send_keys(Keys.ESCAPE).perform()
        driver.get('https://fantasy.top/marketplace')
        time.sleep(5)
        main_element = driver.find_element(By.TAG_NAME, "main")
        main_element.click()
    time.sleep(2)
    
    all_logs = []
    num_iterations = 10
    interval = 3
    
    actions = ActionChains(driver)
    for _ in range(num_iterations):
        logs = driver.get_log("performance")
        all_logs.extend(logs)
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(interval)
    
    websocket_messages = []
    for log in all_logs:
        message = json.loads(log['message'])['message']
        if message['method'] == 'Network.webSocketFrameReceived':
            try:
                payload = json.loads(message['params']['response']['payloadData'])
                if 'payload' in payload and 'data' in payload['payload']:
                    websocket_messages.append(payload)
            except (json.JSONDecodeError, KeyError):
                continue
    
    with open('websocket_messages.json', 'w') as f:
        json.dump(websocket_messages, f, indent=4)
    
    processed_data = []
    for message in websocket_messages:
        if 'payload' in message and 'data' in message['payload']:
            orders = message['payload']['data']['unique_sell_orders_stream']
            for order in orders:
                hero_data = order.get('hero', None)
                if hero_data is not None:  # Ensure hero_data is not None
                    hero_info = {
                        'hero_id': order['hero_id'],
                        'lowest_price': order['lowest_price'],
                        'order_count': order['order_count'],
                        'sell_order_id': order['sell_order_id'],
                        'hero_rarity_index': order['hero_rarity_index'],
                        'gliding_score': order['gliding_score'],
                        'updated_at': order['updated_at'],
                        'hero_followers_count': hero_data.get('followers_count', None),
                        'hero_handle': hero_data.get('handle', None),
                        'hero_name': hero_data.get('name', None),
                        'hero_stars': hero_data.get('stars', None),
                        'current_rank': hero_data.get('current_score', {}).get('current_rank', None) if hero_data.get('current_score') else None,
                        'previous_rank': hero_data.get('current_score', {}).get('previous_rank', None) if hero_data.get('current_score') else None,
                        'views': hero_data.get('current_score', {}).get('views', None) if hero_data.get('current_score') else None,
                        'fantasy_score': hero_data.get('current_score', {}).get('fantasy_score', None) if hero_data.get('current_score') else None
                    }
                    processed_data.append(hero_info)

    
    raw_listings_df = pd.DataFrame(processed_data)
    listings_df = raw_listings_df.drop_duplicates(subset=['hero_id', 'hero_rarity_index'])
    listings_df.loc[:, 'rarity'] = listings_df['hero_rarity_index'].str.split('_').str[1]
    listings_df.loc[:, 'rarity'] = 'rarity' + listings_df['rarity']
    pivot_df = listings_df.pivot_table(
        index='hero_id',
        columns='rarity',
        values=['lowest_price', 'order_count'],
        aggfunc='first'
    )
    pivot_df.columns = [f'{col[1]}_{col[0]}' for col in pivot_df.columns]
    pivot_df.reset_index(inplace=True)
    hero_info_columns = ['hero_id', 'hero_handle', 'hero_name', 'hero_stars', 'hero_followers_count', 
                         'current_rank', 'previous_rank', 'views', 'fantasy_score']
    unique_hero_info = listings_df[hero_info_columns].drop_duplicates(subset=['hero_id'])
    final_df = pd.merge(unique_hero_info, pivot_df, on='hero_id')
    final_df.drop(columns=['hero_name', 'hero_followers_count', 
                           'current_rank', 'previous_rank', 'views', 'fantasy_score'], inplace=True)
    final_df = final_df.rename(columns={'heroId': 'hero_id'})
    return final_df

def send_graphql_request(query=None, variables=None, token=None, request_type='graphql', params=None, cookies=None):
    if request_type == 'graphql':
        payload = json.dumps({
            "query": query,
            "variables": variables
        })
        headers = {
            'accept': '*/*',
            'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
            'authorization': f'Bearer {token}',
            'content-type': 'application/json',
            'origin': 'https://fantasy.top',
            'priority': 'u=1, i',
            'referer': 'https://fantasy.top/',
            'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'cross-site',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
        }
        response = requests.post(URL_GRAPHQL, headers=headers, data=payload, cookies=cookies)
        response.raise_for_status()
        return response.json()
    
    elif request_type == 'rest':
        headers = {
            'accept': '*/*',
            'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
            'authorization': f'Bearer {token}',
            'priority': 'u=1, i',
            'referer': 'https://fantasy.top/marketplace',
            'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
        }
        response = requests.get(URL_REST, params=params, headers=headers, cookies=cookies)
        response.raise_for_status()
        return response.json()

def download_portfolio(token):
    query_get_cards = """
    query GET_CARDS($id: String!, $limit: Int = 100, $offset: Int = 0, $where: i_beta_player_cards_type_bool_exp = {}, $sort_order: String = "") {
      get_player_cards: get_player_cards_new(
        args: {p_owner: $id, p_limit: $limit, p_offset: $offset, p_sort_order: $sort_order}
        where: $where
      ) {
        owner
        hero_rarity_index
        cards_number
        listed_cards_number
        in_deck
        card {
          id
          owner
          gliding_score
          hero_rarity_index
          in_deck
          picture_url
          token_id
          hero_rarity_index
          rarity
          sell_order {
            id
            price_numeric
          }
          hero {
            id
            name
            handle
            profile_image_url_https
            followers_count
            flags {
              flag_id
            }
            stars
            current_score {
              fantasy_score
              views
              current_rank
            }
            trades(
              order_by: {hero_rarity_index: asc, created_at: desc}
              distinct_on: hero_rarity_index
            ) {
              id
              hero_rarity_index
              price
            }
          }
        }
      }
    }
    """
    variables_get_cards = {
        "id": PLAYER_ID,
        "limit": 50,
        "offset": 0,
        "where": {
            "card": {
                "hero": {
                    "_or": [
                        {"name": {"_ilike": "%%"}},
                        {"handle": {"_ilike": "%%"}}
                    ]
                },
                "rarity": {"_in": ["1", "2", "3", "4"]}
            }
        },
        "sort_order": "cards_score"
    }
    
    def extract_portfolio_data(cards_response):
        if 'errors' in cards_response:
            print('Errors:', cards_response['errors'])
            return []
        else:
            cards = cards_response.get('data', {}).get('get_player_cards', [])
            card_list = []
            for card_entry in cards:
                card_data = card_entry['card']
                hero_data = card_data['hero']
                card_info = {
                    'owner': card_entry['owner'],
                    'hero_rarity_index': card_entry['hero_rarity_index'],
                    'cards_number': card_entry['cards_number'],
                    'listed_cards_number': card_entry['listed_cards_number'],
                    'in_deck': card_entry['in_deck'],
                    'card_id': card_data['id'],
                    'card_owner': card_data['owner'],
                    'gliding_score': card_data['gliding_score'],
                    'card_in_deck': card_data['in_deck'],
                    'picture_url': card_data['picture_url'],
                    'token_id': card_data['token_id'],
                    'rarity': card_data['rarity'],
                    'hero_id': hero_data['id'],
                    'hero_name': hero_data['name'],
                    'hero_handle': hero_data['handle'],
                    'hero_profile_image_url': hero_data['profile_image_url_https'],
                    'hero_followers_count': hero_data['followers_count'],
                    'hero_stars': hero_data['stars'],
                    'hero_fantasy_score': hero_data['current_score']['fantasy_score'],
                    'hero_views': hero_data['current_score']['views'],
                    'hero_current_rank': hero_data['current_score']['current_rank'],
                    'hero_trades': [
                        {
                            'trade_id': trade['id'],
                            'hero_rarity_index': trade['hero_rarity_index'],
                            'price': trade['price']
                        } for trade in hero_data['trades']
                    ]
                }
                card_list.append(card_info)
            return card_list
        
    all_cards_list = []
    while True:
        cards_response = send_graphql_request(query_get_cards, variables_get_cards, token)
        portfolio_list = extract_portfolio_data(cards_response)
        if not portfolio_list:
            break
        all_cards_list.extend(portfolio_list)
        variables_get_cards['offset'] += variables_get_cards['limit']
    
    portfolio_df = pd.DataFrame(all_cards_list)
    portfolio_df.drop(columns=['owner', 'card_id', 'card_owner'], inplace=True)
    return portfolio_df

def download_basic_hero_stats(token):
    def extract_heros_data(response_data):
        if 'errors' in response_data:
            print('Errors:', response_data['errors'])
            return []
        heros = response_data.get('data', {}).get('twitter_data_current', [])
        hero_list = []
        for hero_entry in heros:
            hero_data = hero_entry['hero']
            hero_info = {
                'current_rank': hero_entry['current_rank'],
                'previous_rank': hero_entry['previous_rank'],
                'views': hero_entry['views'],
                'tweet_count': hero_entry['tweet_count'],
                'fantasy_score': hero_entry['fantasy_score'],
                'reach': hero_entry['reach'],
                'avg_views': hero_entry['avg_views'],
                'hero_followers_count': hero_data['followers_count'],
                'hero_name': hero_data['name'],
                'hero_handle': hero_data['handle'],
                'hero_profile_image_url': hero_data['profile_image_url_https'],
                'hero_volume': hero_data['volume']['aggregate']['sum']['price'] if hero_data['volume']['aggregate']['sum']['price'] is not None else 0,
                'hero_last_sale_price': hero_data['last_sale'][0]['price'] if hero_data['last_sale'] else None,
                'hero_floor_price': hero_data['floor'][0]['lowest_price'] if hero_data['floor'] else None
            }
            hero_list.append(hero_info)
        return hero_list
    
    query_get_heros_with_stats = """
    query GET_HEROS_WITH_STATS($offset: Int = 0, $limit: Int = 20, $order_by: [twitter_data_current_order_by!] = {current_rank: asc}, $search: String = "") @cached(ttl: 300) {
      twitter_data_current(
        order_by: $order_by
        offset: $offset
        limit: $limit
        where: {hero: {_or: [{name: {_ilike: $search}}, {handle: {_ilike: $search}}], is_pending_hero: {_eq: false}}}
      ) {
        current_rank
        previous_rank
        views
        tweet_count
        fantasy_score
        reach
        avg_views
        hero {
          followers_count
          name
          handle
          profile_image_url_https
          volume: trades_aggregate {
            aggregate {
              sum {
                price
              }
            }
          }
          last_sale: trades(limit: 1) {
            price
          }
          floor: unique_sell_orders(order_by: {lowest_price: asc_nulls_last}, limit: 1) {
            lowest_price
          }
        }
      }
    }
    """
    
    variables_get_heros_with_stats = {
        "offset": 0,
        "order_by": {
            "fantasy_score": "desc"
        },
        "limit": 20,
        "search": "%%"
    }
    
    all_heros_list = []
    while True:
        heros_with_stats_response = send_graphql_request(query=query_get_heros_with_stats, variables=variables_get_heros_with_stats, token=token)
        heros_list = extract_heros_data(heros_with_stats_response)
        if not heros_list:
            break
        all_heros_list.extend(heros_list)
        variables_get_heros_with_stats['offset'] += variables_get_heros_with_stats['limit']
    
    all_heros_df = pd.DataFrame(all_heros_list)
    if 'hero_volume' in all_heros_df.columns:
      all_heros_df['hero_volume'] = all_heros_df['hero_volume'].apply(convert_to_eth)
    all_heros_df['hero_last_sale_price'] = all_heros_df['hero_last_sale_price'].apply(convert_to_eth)
    all_heros_df.drop(columns=['previous_rank', 'hero_last_sale_price', 'hero_floor_price'], inplace=True)
    columns_order = ['current_rank', 'hero_name', 'hero_handle'] + [col for col in all_heros_df.columns if col not in ['current_rank', 'hero_name', 'hero_handle']]
    all_heros_df = all_heros_df[columns_order]
    return all_heros_df

def get_hero_stats(token):
    def adjust_date(created_at_str):
        try:
            created_at = datetime.strptime(created_at_str, '%Y-%m-%dT%H:%M:%S.%f')
        except ValueError:
            created_at = datetime.strptime(created_at_str, '%Y-%m-%dT%H:%M:%S')
        if created_at.time() < datetime.strptime("12:00", "%H:%M").time():
            adjusted_date = created_at - timedelta(days=1)
        else:
            adjusted_date = created_at
        return adjusted_date.date()
    
    def parse_datetime(created_at_str):
        try:
            timestamp = datetime.strptime(created_at_str, '%Y-%m-%dT%H:%M:%S.%f')
        except ValueError:
            timestamp = datetime.strptime(created_at_str, '%Y-%m-%dT%H:%M:%S')
        return timestamp
        
    def get_hero_data(handle, token):
        query_get_hero_by_handle = """
            query GET_HERO_BY_HANDLE($handle: String!) {
          heroes: twitter_data_heroes(where: {handle: {_eq: $handle}}) {
            followers_count
            is_player
            handle
            id
            name
            profile_image_url_https
            distribution_probability {
              inflation_degree
            }
            current_score {
              fantasy_score
              current_rank
              views
            }
            score_history(
              order_by: {created_at: desc}
              limit: 300
              where: {is_gliding24h: {_eq: false}}
            ) {
              id
              fantasy_score
              current_rank
              created_at
            }
            tournament_scores(order_by: {created_at: asc}) {
              id
              current_rank
              views
              created_at
            }
            tweets(order_by: {views: desc}, where: {type: {_nin: ["Retweet", "Reply"]}}) {
              post_id
              bookmarks
              likes
              quotes
              replies
              retweets
              views
              created_at
              type
              fire_score
              impact_score
              health_score
              top_interacting_users
            }
            cards(limit: 1) {
              id
              picture_url
              rarity
            }
            cards_aggregate {
              aggregate {
                count
              }
            }
            trades(limit: 1, order_by: {timestamp: desc}) {
              id
              price
            }
            floor: unique_sell_orders(order_by: {lowest_price: asc_nulls_last}, limit: 1) {
              lowest_price
            }
          }
        }
        """
        variables = {"handle": handle}
        response_data = send_graphql_request(query=query_get_hero_by_handle, variables=variables, token=token)
        if 'errors' in response_data:
            print(f"Error fetching data for handle {handle}: {response_data['errors']}")
            return None
        hero_data = response_data.get('data', {}).get('heroes', [])
        if not hero_data:
            print(f"No data found for handle {handle}")
            return None
        return hero_data[0]
    
    def process_hero_data(hero_data):
        processed_data = {
            "handle": hero_data["handle"],
            "id": hero_data["id"],
            "inflation_degree": hero_data["distribution_probability"]["inflation_degree"] if hero_data["distribution_probability"] else None
        }
        score_history = hero_data.get("score_history", [])
        score_history_dict = {}
        closest_to_midnight = {}
    
        for entry in score_history:
            created_at = entry["created_at"]
            date = adjust_date(created_at)
            timestamp = parse_datetime(created_at)
            closing_score_key = f"{date} Closing Score"
            closing_rank_key = f"{date} Closing Rank"
            midnight = datetime.combine(date, datetime.min.time())
            time_diff = abs((timestamp - midnight).total_seconds())
            if closing_score_key not in closest_to_midnight or time_diff < closest_to_midnight[closing_score_key]:
                closest_to_midnight[closing_score_key] = time_diff
                score_history_dict[closing_score_key] = entry["fantasy_score"]
                score_history_dict[closing_rank_key] = entry["current_rank"]
        processed_data.update(score_history_dict)
        tournament_scores = hero_data.get("tournament_scores", [])
        for entry in tournament_scores:
            try:
                date = datetime.strptime(entry["created_at"], '%Y-%m-%dT%H:%M:%S.%f').date()
            except ValueError:
                date = datetime.strptime(entry["created_at"], '%Y-%m-%d').date()
            tournament_rank_key = f"{date} Tournament Rank"
            processed_data[tournament_rank_key] = entry["current_rank"]
        return processed_data

    basic_hero_stats_df = pd.read_csv(get_latest_hero_stats_file(DATA_FOLDER))  
    all_hero_data = []
    for handle in tqdm(basic_hero_stats_df["hero_handle"], desc="Processing heroes"):
        hero_data = get_hero_data(handle, token)
        if hero_data:
            processed_data = process_hero_data(hero_data)
            all_hero_data.append(processed_data)
        time.sleep(random.uniform(1, 3))
    
    hero_scores = pd.DataFrame(all_hero_data)
    hero_scores = hero_scores.rename(columns={'handle': 'hero_handle', 'id': 'hero_id'})
    return hero_scores

def get_hero_supply(hero_stats_df, token):
    query_get_supply_per_hero_id = """
    query GET_SUPPLY_PER_HERO_ID($heroId: String!) @cached(ttl: 3600) {
      rarity1Count: indexer_cards_aggregate(
        where: {hero_id: {_eq: $heroId}, rarity: {_eq: 1}, owner: {_neq: "0x0000000000000000000000000000000000000000"}}
      ) {
        aggregate {
          count
        }
      }
      rarity2Count: indexer_cards_aggregate(
        where: {hero_id: {_eq: $heroId}, rarity: {_eq: 2}, owner: {_neq: "0x0000000000000000000000000000000000000000"}}
      ) {
        aggregate {
          count
        }
      }
      rarity3Count: indexer_cards_aggregate(
        where: {hero_id: {_eq: $heroId}, rarity: {_eq: 3}, owner: {_neq: "0x0000000000000000000000000000000000000000"}}
      ) {
        aggregate {
          count
        }
      }
      rarity4Count: indexer_cards_aggregate(
        where: {hero_id: {_eq: $heroId}, rarity: {_eq: 4}, owner: {_neq: "0x0000000000000000000000000000000000000000"}}
      ) {
        aggregate {
          count
        }
      }
      burnedCardsCount: indexer_cards_aggregate(
        where: {hero_id: {_eq: $heroId}, owner: {_eq: "0x0000000000000000000000000000000000000000"}}
      ) {
        aggregate {
          count
        }
      }
      utilityCount: indexer_cards_aggregate(
        where: {hero_id: {_eq: $heroId}, in_deck: {_eq: true}, owner: {_neq: "0x0000000000000000000000000000000000000000"}}
      ) {
        aggregate {
          count
        }
      }
    }
    """
    
    def process_get_supply_per_hero_id(response, hero_id):
        if 'errors' in response:
            print('Errors:', response['errors'])
            return pd.DataFrame()
        data = response.get('data', {})
        supply_data = {
            'heroId': hero_id,
            'rarity1Count': data['rarity1Count']['aggregate']['count'],
            'rarity2Count': data['rarity2Count']['aggregate']['count'],
            'rarity3Count': data['rarity3Count']['aggregate']['count'],
            'rarity4Count': data['rarity4Count']['aggregate']['count'],
            'burnedCardsCount': data['burnedCardsCount']['aggregate']['count'],
            'utilityCount': data['utilityCount']['aggregate']['count']
        }
        return pd.DataFrame([supply_data])
    
    def get_supply_per_hero_id(url, query, hero_ids, token, delay=1, max_retries=3):
        all_supplies = []
        total_heroes = len(hero_ids)
        with tqdm(total=total_heroes, desc="Fetching hero data") as pbar:
            for index, hero_id in enumerate(hero_ids):
                variables = {"heroId": str(hero_id)}
                retries = 0
                while retries < max_retries:
                    try:
                        status_message = f"Fetching data for hero {hero_id} ({index+1}/{total_heroes}), attempt {retries+1}"
                        sys.stdout.write(status_message)
                        sys.stdout.flush()
                        response = send_graphql_request(query=query, variables=variables, token=token)
                        supply_df = process_get_supply_per_hero_id(response, hero_id)
                        all_supplies.append(supply_df)
                        sys.stdout.write(f"Successfully fetched data for hero {hero_id}          \n")
                        sys.stdout.flush()
                        time.sleep(delay)
                        break
                    except Exception as e:
                        sys.stdout.write(f"Error fetching data for hero {hero_id}: {e}          \r")
                        sys.stdout.flush()
                        retries += 1
                        time.sleep(delay * retries)
                        if retries >= max_retries:
                            sys.stdout.write(f"Failed to fetch data for hero {hero_id} after {max_retries} attempts\n")
                            sys.stdout.flush()
                pbar.update(1)
        return pd.concat(all_supplies, ignore_index=True)
    
    hero_ids = hero_stats_df['hero_id'].tolist()
    all_hero_supplies_df = get_supply_per_hero_id(URL_GRAPHQL, query_get_supply_per_hero_id, hero_ids, token)
    all_hero_supplies_df = all_hero_supplies_df.rename(columns={'heroId': 'hero_id'})
    return all_hero_supplies_df

def get_bids(hero_stats_df, token, cookies):
    def get_highest_bids_for_hero(hero_id, token, cookies, delay=2, max_retries=3):
        hero_bids = {'hero_id': hero_id}
        for rarity in range(1, 5):
            params = {
                'hero_id': hero_id,
                'rarity': rarity,
                'include_orderbook': 'true',
                'include_personal_bids': 'true',
            }
            retries = 0
            while retries < max_retries:
                try:
                    status_message = f"Fetching data for hero {hero_id} rarity {rarity}, attempt {retries + 1}\r"
                    sys.stdout.write(status_message)
                    sys.stdout.flush()
                    response = send_graphql_request(request_type='rest', params=params, token=token, cookies=cookies)
                    highest_bid = 0
                    if response.get('orderbook_bids'):
                        highest_bid = max(int(bid['price']) for bid in response['orderbook_bids'])
                        highest_bid /= 1e18
                    hero_bids[f'rarity{rarity}HighestBid'] = highest_bid
                    break
                except Exception as e:
                    sys.stdout.write(f"Error fetching data for hero {hero_id} rarity {rarity}: {e}, attempt {retries + 1}\r")
                    sys.stdout.flush()
                    retries += 1
                    time.sleep(delay * retries)
            if retries >= max_retries:
                sys.stdout.write(f"Failed to fetch data for hero {hero_id} rarity {rarity} after {max_retries} attempts\n")
                sys.stdout.flush()
                hero_bids[f'rarity{rarity}HighestBid'] = None
        return hero_bids
    
    def collect_highest_bids(ref_df, token, cookies, delay=7, max_retries=3):
        data = []
        total_heroes = len(ref_df['hero_id'])
        for index, hero_id in enumerate(ref_df['hero_id']):
            status_message = f"Processing hero {hero_id} ({index + 1}/{total_heroes})...\r"
            sys.stdout.write(status_message)
            sys.stdout.flush()
            hero_bids = get_highest_bids_for_hero(hero_id, token, cookies, delay, max_retries)
            data.append(hero_bids)
            sys.stdout.write(f"Completed {index + 1} out of {total_heroes}\n")
            sys.stdout.flush()
            time.sleep(delay)
        highest_bids_df = pd.DataFrame(data)
        return highest_bids_df
    
    highest_bids_df = collect_highest_bids(hero_stats_df, token, cookies)
    return highest_bids_df

def get_last_trades(token):
    query_get_last_trade = """
    query GET_LAST_TRADE {
      indexer_trades(
        distinct_on: hero_rarity_index
        order_by: {hero_rarity_index: asc, timestamp: desc}
      ) {
        id
        hero_rarity_index
        price
        timestamp
      }
    }
    """
    
    def process_get_last_trade(response):
        if 'errors' in response:
            print('Errors:', response['errors'])
            return pd.DataFrame()
        trades = response.get('data', {}).get('indexer_trades', [])
        last_trade_data = []
        for trade in trades:
            hero_id, rarity = trade['hero_rarity_index'].split('_')
            last_trade_data.append({
                'heroId': hero_id,
                'rarity': f'rarity{rarity}',
                'lastSalePrice': convert_to_eth(trade['price']),
                'lastSaleTime': trade['timestamp']
            })
        return pd.DataFrame(last_trade_data)
    
    def get_last_trade(url, query, token):
        response = send_graphql_request(query=query, token=token)
        last_trade_df = process_get_last_trade(response)
        return last_trade_df
    
    last_trade_df = get_last_trade(URL_GRAPHQL, query_get_last_trade, token)
    pivoted_df = last_trade_df.pivot(index='heroId', columns='rarity', values=['lastSalePrice', 'lastSaleTime'])
    pivoted_df.columns = [f'{col[1]}{col[0]}' for col in pivoted_df.columns]
    pivoted_df.reset_index(inplace=True)
    pivoted_df = pivoted_df.rename(columns={'heroId': 'hero_id'})
    return pivoted_df

def get_latest_hero_stats_file(directory, prefix='hero_stats'):
    files = glob.glob(os.path.join(directory, f'{prefix}*.csv'))
    if not files:
        raise FileNotFoundError(f"No files starting with '{prefix}' found in {directory}")
    latest_file = max(files, key=os.path.getmtime)
    return latest_file

# Main Execution Function




In [17]:

driver, token = login()
cookies = {cookie['name']: cookie['value'] for cookie in driver.get_cookies()}


Accept button did not appear.
Logged In Successfully


In [15]:

hero_stats_df = pd.read_csv(get_latest_hero_stats_file(DATA_FOLDER))

save_df_as_csv(print_runtime(get_hero_stats, token), 'hero_stats')
save_df_as_csv(print_runtime(download_portfolio, token), 'portfolio')
save_df_as_csv(print_runtime(download_basic_hero_stats, token), 'basic_hero_stats')
save_df_as_csv(print_runtime(download_listings, driver), 'listings')
save_df_as_csv(print_runtime(get_last_trades, token), 'last_trades')
save_df_as_csv(print_runtime(get_hero_supply, hero_stats_df, token), 'hero_card_supply')
save_df_as_csv(print_runtime(get_bids, hero_stats_df, token, cookies), 'bids')


Calling get_hero_stats


Processing heroes: 100%|██████████| 177/177 [16:54<00:00,  5.73s/it]


get_hero_stats took 1014.3997 seconds to execute
DataFrame saved as data\hero_stats_240815_2023.csv
Calling download_portfolio
download_portfolio took 38.3169 seconds to execute
DataFrame saved as data\portfolio_240815_2024.csv
Calling download_basic_hero_stats
download_basic_hero_stats took 37.0860 seconds to execute
DataFrame saved as data\basic_hero_stats_240815_2024.csv
Calling download_listings


C:\Users\beuzi\AppData\Local\Temp\ipykernel_5964\1046994117.py:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_df.loc[:, 'rarity'] = listings_df['hero_rarity_index'].str.split('_').str[1]


download_listings took 43.5247 seconds to execute
DataFrame saved as data\listings_240815_2025.csv
Calling get_last_trades
get_last_trades took 4.5994 seconds to execute
DataFrame saved as data\last_trades_240815_2025.csv
Calling get_hero_supply


...

Fetching hero data:  99%|█████████▉| 176/177 [03:25<00:01,  1.18s/it]

Fetching data for hero 1372325396461580292 (177/177), attempt 1Successfully fetched data for hero 1372325396461580292          


Fetching hero data: 100%|██████████| 177/177 [03:26<00:00,  1.16s/it]

get_hero_supply took 206.1953 seconds to execute
DataFrame saved as data\hero_card_supply_240815_2029.csv
Calling get_bids


Completed 1 out of 177 1356434353623093249 rarity 4, attempt 1
Completed 2 out of 177 262018615 rarity 4, attempt 1
Completed 3 out of 177 982070158125682688 rarity 4, attempt 1
Failed to fetch data for hero 973261472 rarity 4 after 3 attempts: Too Many Requests for url: https://fantasy.top/api/bids/get-bid-orders?hero_id=973261472&rarity=4&include_orderbook=true&include_personal_bids=true, attempt 3
Completed 4 out of 177
Completed 5 out of 177 1441835930889818113 rarity 4, attempt 1
Completed 6 out of 177 906234475604037637 rarity 4, attempt 1
Completed 7 out of 177 588569122 rarity 4, attempt 1
Failed to fetch data for hero 965651 rarity 4 after 3 attempts: Too Many Requests for url: https://fantasy.top/api/bids/get-bid-orders?hero_id=965651&rarity=4&include_orderbook=true&include_personal_bids=true, attempt 3
Completed 8 out of 177
Completed 9 out of 177 279828819 rarity 4, attempt 1
Completed 10 out of 1771466010689726783491 rarity 4, attempt 1
Completed 11 out of 1771446541960181

## Imports

In [2]:
#####
# At the moment this loads of fantasy and helps get a list of heroes
# If the tutorials pops up need to manually skip it
# Can probably be avoided if use cookies
#

import os
import re
import csv
import sys
import json
import time
import random
import inspect # for saving dataframes using the name
import pickle # Cookie Manager
from tqdm import tqdm # status bar
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from fake_useragent import UserAgent
import pyautogui



## Cookies

In [3]:
def save_cookies(driver, filepath='cookies.pkl'):
    with open(filepath, 'wb') as file:
        pickle.dump(driver.get_cookies(), file)

def load_cookies(driver, filepath='cookies.pkl'):
    try:
        with open(filepath, 'rb') as file:
            cookies = pickle.load(file)
            for cookie in cookies:
                driver.add_cookie(cookie)
    except FileNotFoundError:
        print("Cookies file not found. Proceeding without loading cookies.")


## Login in to Fantasy Top

In [4]:
from webdriver_manager.chrome import ChromeDriverManager
import os

# Find the path where ChromeDriverManager installs ChromeDriver
driver_path = ChromeDriverManager().install()

# Manually delete the installed ChromeDriver
os.remove(driver_path)

# Reinstall ChromeDriver
driver_path = ChromeDriverManager().install()

print(f"Reinstalled ChromeDriver at: {driver_path}")

Reinstalled ChromeDriver at: C:\Users\beuzi\.wdm\drivers\chromedriver\win64\127.0.6533.119\chromedriver-win32/THIRD_PARTY_NOTICES.chromedriver


In [5]:
# def random_sleep(min_time=1, max_time=3):
#     time.sleep(random.uniform(min_time, max_time))

# def move_mouse_randomly():
#     for _ in range(random.randint(1, 3)):
#         x, y = random.randint(100, 500), random.randint(100, 500)
#         pyautogui.moveTo(x, y, duration=random.uniform(0.1, 0.5))

def get_random_user_agent():
    ua = UserAgent(platforms='pc')
    return ua.random

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument(f"user-agent={get_random_user_agent()}")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--disable-notifications")
    options.add_argument("--disable-popup-blocking")
    options.add_argument("--auto-open-devtools-for-tabs")
    # Enable performance logging
    perf_log_prefs = {
        'enableNetwork': True,
        'enablePage': False,
    }
    options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
    options.add_argument("--window-size=1920,1080")
    # options.add_argument("--headless")
    
    driver = webdriver.Chrome(options=options)
    
    return driver

def login_to_fantasy(driver, username, password):
    driver.get("https://www.fantasy.top/home")
    
    wait = WebDriverWait(driver, 10)
    
    # Click initial button
    button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[style*='background: linear-gradient'][class*='rounded-md']")))
    button.click()
    
    # Click Twitter button in modal
    modal_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Twitter')]")))
    modal_button.click()
    
    time.sleep(2)
    actions = ActionChains(driver)
    actions.send_keys(Keys.ESCAPE).perform()
    
    username_input = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='text'][autocomplete='username']")))
    username_input.send_keys(username)
    
    next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Next']/ancestor::button")))
    next_button.click()
    
    password_input = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password'][type='password']")))
    password_input.send_keys(password)
    
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='LoginForm_Login_Button']")))
    login_button.click()
    
    # Handle pop-up, authorization, and acceptance
    close_popup_if_appears(driver)
    authorize_if_appears(driver)
    accept_terms_if_appears(driver)
    
    # Load main page and handle additional elements
    driver.get("https://www.fantasy.top/home")
    # dismiss_notification_if_appears(driver)
    # skip_tutorial_if_appears(driver)
    print("Logged In Successfully")

def close_popup_if_appears(driver):
    try:
        close_popup_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='xMigrationBottomBar']"))
        )
        close_popup_button.click()
    except TimeoutException:
        print("Close popup button did not appear.")

def authorize_if_appears(driver):
    try:
        authorize_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='OAuth_Consent_Button']"))
        )
        authorize_button.click()
    except TimeoutException:
        print("Authorize app button did not appear.")

def accept_terms_if_appears(driver):
    try:
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.sc-fqkvVR.sc-iGgWBj.dQeymh.httOiR"))
        )
        accept_button.click()
    except TimeoutException:
        print("Accept button did not appear.")

# def dismiss_notification_if_appears(driver):
#     try:
#         element = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, "//p[contains(@class, 'link') and contains(text(), \"Don't show again\")]"))
#         )
#         element.click()
#         print("Notification dismissed")
#     except Exception as e:
#         print("Notification not found or not clickable:", e)

# def skip_tutorial_if_appears(driver):
#     try:
#         skip_button = WebDriverWait(driver, 10).until(
#             EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-test-id='button-skip']"))
#         )
#         skip_button.click()
#     except TimeoutException:
#         print("Skip button did not appear.")

def scroll_to_end_of_page(driver):
    # Get the initial page height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for new page segment to load
        time.sleep(2)  # You can adjust the sleep time if needed

        # Calculate new page height and compare with last height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


def login():
    username = "MissPecuIiar"  # Replace with actual username
    password = "Dublin84"      # Replace with actual password
    
    driver = setup_driver()
    
    driver.get("https://www.fantasy.top/home")  # Load the target site first
    
    load_cookies(driver)  # Load cookies for the correct domain
    driver.refresh()  # Refresh to apply cookies
    
    login_to_fantasy(driver, username, password)
    token = driver.execute_script("return localStorage.getItem('jwtToken');")
    save_cookies(driver)

    time.sleep(2)

    actions = ActionChains(driver)
    actions.send_keys(Keys.ESCAPE).perform()
    
    
    return driver, token

# Call the login function and store the driver and token
driver, token = login()

# Retrieve the token if needed (already done in login function)
selenium_cookies = driver.get_cookies()
cookies = {cookie['name']: cookie['value'] for cookie in selenium_cookies}




Accept button did not appear.
Logged In Successfully


In [6]:
token = driver.execute_script("return localStorage.getItem('jwtToken');")

In [12]:
import requests

URL_REST = "https://fantasy.top/api/bids/get-bid-orders"

def send_graphql_request(request_type='rest', params=None, token=None, cookies=None):
    headers = {
        'accept': '*/*',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'authorization': f'Bearer {token}',
        'priority': 'u=1, i',
        'referer': 'https://fantasy.top/marketplace',
        'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    }
    response = requests.get(URL_REST, params=params, headers=headers, cookies=cookies)
    response.raise_for_status()
    return response.json()

def get_bids_for_multiple_heroes(hero_ids, token, cookies, rarity=1):
    params = {
        'hero_ids': ','.join(hero_ids),  # Or try different separators like ';' or '|'
        'rarity': rarity,
        'include_orderbook': 'true',
        'include_personal_bids': 'true',
    }
    response = send_graphql_request(request_type='rest', params=params, token=token, cookies=cookies)
    return response

def test_batched_requests():
    # Sample hero_ids for testing - Replace with actual hero_ids
    hero_ids = ["hero_id_1", "hero_id_2", "hero_id_3", "hero_id_4", "hero_id_5"]

    try:
        response = get_bids_for_multiple_heroes(hero_ids, token, cookies)
        print("Response from API:")
        print(response)
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    test_batched_requests()

An error occurred: 500 Server Error: Internal Server Error for url: https://fantasy.top/api/bids/get-bid-orders?hero_ids=hero_id_1%2Chero_id_2%2Chero_id_3%2Chero_id_4%2Chero_id_5&rarity=1&include_orderbook=true&include_personal_bids=true


## Utility Functions

In [80]:
def convert_to_eth(value):
    return int(value) / 1e18 if value is not None else None



### Save df to CSV 

In [81]:
def save_df_as_csv(df, filename, folder="data"):
    # Generate the timestamp
    timestamp = datetime.now().strftime('%y%m%d_%H%M')
    # Append timestamp to the filename
    filename_with_timestamp = f"{filename}_{timestamp}.csv"
    # Create the folder if it doesn't exist
    if not os.path.exists(folder):
        os.makedirs(folder)
    # Full path
    full_path = os.path.join(folder, filename_with_timestamp)
    # Save the DataFrame as a CSV
    df.to_csv(full_path, index=False)
    print(f"DataFrame saved as {full_path}")

In [82]:
def print_runtime(func) -> None:
    """
    Measures and prints the runtime of a given function.

    Parameters:
    func (Callable): The function whose runtime is to be measured.

    Returns:
    None
    """
    print(f'Calling {func.__name__}')
    start_time = time.time()
    func()
    end_time = time.time()
    runtime = end_time - start_time
    print(f'{func.__name__} took {runtime:.4f} seconds to execute')

## Websockets Market Data

In [93]:
def download_listings():
    driver.get('https://fantasy.top/marketplace')
    time.sleep(5)
    # Click on the specified element to ensure the page is in the correct state for scrolling
    actions = ActionChains(driver)
    actions.send_keys(Keys.ESCAPE).perform()
    try:
        main_element = driver.find_element(By.TAG_NAME, "main")
        main_element.click()
    except ElementClickInterceptedException as e:
        actions.send_keys(Keys.ESCAPE).perform()
        main_element.click()
        driver.get('https://fantasy.top/marketplace')
        time.sleep(5)
        main_element = driver.find_element(By.TAG_NAME, "main")
        main_element.click()
    time.sleep(2)
    
    
    
    
    # Initialize a list to store all logs
    all_logs = []
    
    # Define the number of iterations and the interval between them
    num_iterations = 10
    interval = 3  # in seconds
    
    
    
    actions = ActionChains(driver)
    # Continuously capture logs and scroll down
    for _ in range(num_iterations):
        # Capture logs
        logs = driver.get_log("performance")
        all_logs.extend(logs)
        
        # Scroll down by sending PAGE_DOWN key using ActionChains
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(interval)  # Adjust the interval as needed
    
    # Close the browser
    # driver.quit()
    
    # Filter WebSocket messages
    websocket_messages = []
    
    for log in all_logs:
        message = json.loads(log['message'])['message']
        if message['method'] == 'Network.webSocketFrameReceived':
            try:
                payload = json.loads(message['params']['response']['payloadData'])
                if 'payload' in payload and 'data' in payload['payload']:
                    websocket_messages.append(payload)
            except (json.JSONDecodeError, KeyError) as e:
                print(f"Error processing message: {e}")
                continue
    
    # Save the captured WebSocket messages to a file for further analysis
    with open('websocket_messages.json', 'w') as f:
        json.dump(websocket_messages, f, indent=4)
    
    # Process the WebSocket messages
    processed_data = []
    
    for message in websocket_messages:
        if 'payload' in message and 'data' in message['payload']:
            orders = message['payload']['data']['unique_sell_orders_stream']
            for order in orders:
                hero_info = {
                    'hero_id': order['hero_id'],
                    'lowest_price': order['lowest_price'],
                    'order_count': order['order_count'],
                    'sell_order_id': order['sell_order_id'],
                    'hero_rarity_index': order['hero_rarity_index'],
                    'gliding_score': order['gliding_score'],
                    'updated_at': order['updated_at'],
                    'hero_followers_count': order['hero']['followers_count'],
                    'hero_handle': order['hero']['handle'],
                    'hero_name': order['hero']['name'],
                    'hero_stars': order['hero']['stars'],
                    'current_rank': order['hero']['current_score']['current_rank'],
                    'previous_rank': order['hero']['current_score']['previous_rank'],
                    'views': order['hero']['current_score']['views'],
                    'fantasy_score': order['hero']['current_score']['fantasy_score']
                }
                processed_data.append(hero_info)
    
    # Convert the processed data into a DataFrame
    raw_listings_df = pd.DataFrame(processed_data)
    
    listings_df = raw_listings_df.drop_duplicates(subset=['hero_id', 'hero_rarity_index'])

    # Step 1: Extract rarity from `hero_rarity_index`
    listings_df.loc[:, 'rarity'] = listings_df['hero_rarity_index'].str.split('_').str[1]
    listings_df.loc[:, 'rarity'] = 'rarity' + listings_df['rarity']
    
    pivot_df = listings_df.pivot_table(
        index='hero_id',
        columns='rarity',
        values=['lowest_price', 'order_count'],
        aggfunc='first'
    )
    
    # Flatten the multi-index columns
    pivot_df.columns = [f'{col[1]}_{col[0]}' for col in pivot_df.columns]
    
    # Reset the index to make `hero_id` a column
    pivot_df.reset_index(inplace=True)
    
    # Step 3: Merge the pivoted DataFrame with the unique hero information
    hero_info_columns = ['hero_id', 'hero_handle', 'hero_name', 'hero_stars', 'hero_followers_count', 
                         'current_rank', 'previous_rank', 'views', 'fantasy_score']
    
    # Drop duplicates to get unique hero information
    unique_hero_info = listings_df[hero_info_columns].drop_duplicates(subset=['hero_id'])
    
    # Merge the pivoted data with the unique hero information
    final_df = pd.merge(unique_hero_info, pivot_df, on='hero_id')

    columns_to_remove = [
    'hero_name', 'hero_followers_count', 
    'current_rank', 'previous_rank', 'views', 'fantasy_score'
]
    final_df.drop(columns=columns_to_remove, inplace=True)
    
    final_df = final_df.rename(columns={'heroId': 'hero_id'})

    return final_df 

## API 

### Params

In [19]:
import requests
import json

url_graphql = "https://fantasy-top.hasura.app/v1/graphql"
url_rest = "https://fantasy.top/api/bids/get-bid-orders"

def send_graphql_request(url=url_graphql, query=None, variables=None, token=None, request_type='graphql', params=None, cookies=None):
    if request_type == 'graphql':
        payload = json.dumps({
            "query": query,
            "variables": variables
        })
        
        headers = {
            'accept': '*/*',
            'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
            'authorization': f'Bearer {token}',
            'content-type': 'application/json',
            'origin': 'https://fantasy.top',
            'priority': 'u=1, i',
            'referer': 'https://fantasy.top/',
            'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'cross-site',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
        }
        
        response = requests.post(url, headers=headers, data=payload, cookies=cookies)
        response.raise_for_status()
        return response.json()
    
    elif request_type == 'rest':
        headers = {
            'accept': '*/*',
            'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
            'authorization': f'Bearer {token}',
            'priority': 'u=1, i',
            'referer': 'https://fantasy.top/marketplace',
            'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
        }
        
        response = requests.get(url, params=params, headers=headers, cookies=cookies)
        response.raise_for_status()
        return response.json()





### Get Own Cards

In [85]:

def download_portfolio():
    # Query 1: GET_CARDS
    query_get_cards = """
    query GET_CARDS($id: String!, $limit: Int = 100, $offset: Int = 0, $where: i_beta_player_cards_type_bool_exp = {}, $sort_order: String = "") {
      get_player_cards: get_player_cards_new(
        args: {p_owner: $id, p_limit: $limit, p_offset: $offset, p_sort_order: $sort_order}
        where: $where
      ) {
        owner
        hero_rarity_index
        cards_number
        listed_cards_number
        in_deck
        card {
          id
          owner
          gliding_score
          hero_rarity_index
          in_deck
          picture_url
          token_id
          hero_rarity_index
          rarity
          sell_order {
            id
            price_numeric
          }
          hero {
            id
            name
            handle
            profile_image_url_https
            followers_count
            flags {
              flag_id
            }
            stars
            current_score {
              fantasy_score
              views
              current_rank
            }
            trades(
              order_by: {hero_rarity_index: asc, created_at: desc}
              distinct_on: hero_rarity_index
            ) {
              id
              hero_rarity_index
              price
            }
          }
        }
      }
    }
    """
    
    variables_get_cards = {
        "id": "0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a",
        "limit": 50,
        "offset": 0,
        "where": {
            "card": {
                "hero": {
                    "_or": [
                        {"name": {"_ilike": "%%"}},
                        {"handle": {"_ilike": "%%"}}
                    ]
                },
                "rarity": {"_in": ["1", "2", "3", "4"]}
            }
        },
        "sort_order": "cards_score"
    }
    
    
    def extract_portfolio_data(cards_response):
        # Check for errors in the response
        if 'errors' in cards_response:
            print('Errors:', cards_response['errors'])
            return []
        else:
            # Extract the relevant data
            cards = cards_response.get('data', {}).get('get_player_cards', [])
        
            # Process the extracted data
            card_list = []
            for card_entry in cards:
                card_data = card_entry['card']
                hero_data = card_data['hero']
                
                card_info = {
                    'owner': card_entry['owner'],
                    'hero_rarity_index': card_entry['hero_rarity_index'],
                    'cards_number': card_entry['cards_number'],
                    'listed_cards_number': card_entry['listed_cards_number'],
                    'in_deck': card_entry['in_deck'],
                    'card_id': card_data['id'],
                    'card_owner': card_data['owner'],
                    'gliding_score': card_data['gliding_score'],
                    'card_in_deck': card_data['in_deck'],
                    'picture_url': card_data['picture_url'],
                    'token_id': card_data['token_id'],
                    'rarity': card_data['rarity'],
                    'hero_id': hero_data['id'],
                    'hero_name': hero_data['name'],
                    'hero_handle': hero_data['handle'],
                    'hero_profile_image_url': hero_data['profile_image_url_https'],
                    'hero_followers_count': hero_data['followers_count'],
                    'hero_stars': hero_data['stars'],
                    'hero_fantasy_score': hero_data['current_score']['fantasy_score'],
                    'hero_views': hero_data['current_score']['views'],
                    'hero_current_rank': hero_data['current_score']['current_rank'],
                    'hero_trades': [
                        {
                            'trade_id': trade['id'],
                            'hero_rarity_index': trade['hero_rarity_index'],
                            'price': trade['price']
                        } for trade in hero_data['trades']
                    ]
                }
                
                card_list.append(card_info)
        
            # Return the list of card data
            return card_list
        
    
    all_cards_list = []
    while True:
        # Make the request
        cards_response = send_graphql_request(url_graphql, query_get_cards, variables_get_cards, token)
        
        # Extract data
        portfolio_list = extract_portfolio_data(cards_response)
        
        # If no more data is returned, break the loop
        if not portfolio_list:
            break
        
        # Append the data to the main list
        all_cards_list.extend(portfolio_list)
        
        # Update the offset for the next request
        variables_get_cards['offset'] += variables_get_cards['limit']
    
    # Create a DataFrame from the aggregated list
    portfolio_df = pd.DataFrame(all_cards_list)

    portfolio_df.drop(columns = ['owner', 'card_id', 'card_owner'], inplace=True)

    return portfolio_df


### Get All Heroes Stats

In [86]:
def download_basic_hero_stats():

    # Function to extract data from the response and convert it to a DataFrame
    def extract_heros_data(response_data):
        if 'errors' in response_data:
            print('Errors:', response_data['errors'])
            return []
    
        heros = response_data.get('data', {}).get('twitter_data_current', [])
    
        hero_list = []
        for hero_entry in heros:
            hero_data = hero_entry['hero']
            
            hero_info = {
                'current_rank': hero_entry['current_rank'],
                'previous_rank': hero_entry['previous_rank'],
                'views': hero_entry['views'],
                'tweet_count': hero_entry['tweet_count'],
                'fantasy_score': hero_entry['fantasy_score'],
                'reach': hero_entry['reach'],
                'avg_views': hero_entry['avg_views'],
                'hero_followers_count': hero_data['followers_count'],
                'hero_name': hero_data['name'],
                'hero_handle': hero_data['handle'],
                'hero_profile_image_url': hero_data['profile_image_url_https'],
                'hero_volume': hero_data['volume']['aggregate']['sum']['price'] if hero_data['volume']['aggregate']['sum']['price'] is not None else 0,
                'hero_last_sale_price': hero_data['last_sale'][0]['price'] if hero_data['last_sale'] else None,
                'hero_floor_price': hero_data['floor'][0]['lowest_price'] if hero_data['floor'] else None
            }
            
            hero_list.append(hero_info)
        
        return hero_list
    
    # Define the query and initial variables
    query_get_heros_with_stats = """
    query GET_HEROS_WITH_STATS($offset: Int = 0, $limit: Int = 20, $order_by: [twitter_data_current_order_by!] = {current_rank: asc}, $search: String = "") @cached(ttl: 300) {
      twitter_data_current(
        order_by: $order_by
        offset: $offset
        limit: $limit
        where: {hero: {_or: [{name: {_ilike: $search}}, {handle: {_ilike: $search}}], is_pending_hero: {_eq: false}}, is_gliding24h: {_eq: false}}
      ) {
        current_rank
        previous_rank
        views
        tweet_count
        fantasy_score
        reach
        avg_views
        hero {
          followers_count
          name
          handle
          profile_image_url_https
          volume: trades_aggregate {
            aggregate {
              sum {
                price
              }
            }
          }
          last_sale: trades(limit: 1) {
            price
          }
          floor: unique_sell_orders(order_by: {lowest_price: asc_nulls_last}, limit: 1) {
            lowest_price
          }
        }
      }
    }
    """
    
    # Variables for pagination
    variables_get_heros_with_stats = {
        "offset": 0,
        "order_by": {
            "fantasy_score": "desc"
        },
        "limit": 20,  # Adjust the limit if necessary
        "search": "%%"
    }
    
    # Pagination logic
    all_heros_list = []
    while True:
        # Make the request
        heros_with_stats_response = send_graphql_request(url_graphql, query_get_heros_with_stats, variables_get_heros_with_stats, token)
        
        # Extract data
        heros_list = extract_heros_data(heros_with_stats_response)
        
        # If no more data is returned, break the loop
        if not heros_list:
            break
        
        # Append the data to the main list
        all_heros_list.extend(heros_list)
        
        # Update the offset for the next request
        variables_get_heros_with_stats['offset'] += variables_get_heros_with_stats['limit']
    
    # Create a DataFrame from the aggregated list
    all_heros_df = pd.DataFrame(all_heros_list)

    
    all_heros_df['hero_volume'] = all_heros_df['hero_volume'].apply(convert_to_eth)
    all_heros_df['hero_last_sale_price'] = all_heros_df['hero_last_sale_price'].apply(convert_to_eth)

    # List of columns to remove
    columns_to_remove = ['previous_rank', 'hero_last_sale_price', 'hero_floor_price']

    all_heros_df.drop(columns=columns_to_remove, inplace=True)

    # Reorder the columns to move 'hero_name' and 'hero_handle' to the first two positions
    columns_order = ['current_rank', 'hero_name', 'hero_handle'] + [col for col in all_heros_df.columns if col not in ['current_rank', 'hero_name', 'hero_handle']]

    all_heros_df = all_heros_df[columns_order]

    return all_heros_df



### Hero Scores - Get hero by handle - Get previous Tournament Scores Also Twitter Data available but not yet utilised



In [87]:
def get_hero_stats():
     
    def adjust_date(created_at_str):
        try:
            created_at = datetime.strptime(created_at_str, '%Y-%m-%dT%H:%M:%S.%f')
        except ValueError:
            created_at = datetime.strptime(created_at_str, '%Y-%m-%dT%H:%M:%S')
        if created_at.time() < datetime.strptime("12:00", "%H:%M").time():  # Check if the time is before noon
            adjusted_date = created_at - timedelta(days=1)
        else:
            adjusted_date = created_at
        return adjusted_date.date()
    
    def parse_datetime(created_at_str):
        try:
            timestamp = datetime.strptime(created_at_str, '%Y-%m-%dT%H:%M:%S.%f')
        except ValueError:
            timestamp = datetime.strptime(created_at_str, '%Y-%m-%dT%H:%M:%S')
        return timestamp
        
    
    def get_hero_data(handle, url, token):
        query_get_hero_by_handle = """
            query GET_HERO_BY_HANDLE($handle: String!) {
          heroes: twitter_data_heroes(where: {handle: {_eq: $handle}}) {
            followers_count
            is_player
            handle
            id
            name
            profile_image_url_https
            distribution_probability {
              inflation_degree
            }
            current_score {
              fantasy_score
              current_rank
              views
            }
            score_history(
              order_by: {created_at: desc}
              limit: 300
              where: {is_gliding24h: {_eq: false}}
            ) {
              id
              fantasy_score
              current_rank
              created_at
            }
            tournament_scores(order_by: {created_at: asc}) {
              id
              current_rank
              views
              created_at
            }
            tweets(order_by: {views: desc}, where: {type: {_nin: ["Retweet", "Reply"]}}) {
              post_id
              bookmarks
              likes
              quotes
              replies
              retweets
              views
              created_at
              type
              fire_score
              impact_score
              health_score
              top_interacting_users
            }
            cards(limit: 1) {
              id
              picture_url
              rarity
            }
            cards_aggregate {
              aggregate {
                count
              }
            }
            trades(limit: 1, order_by: {timestamp: desc}) {
              id
              price
            }
            floor: unique_sell_orders(order_by: {lowest_price: asc_nulls_last}, limit: 1) {
              lowest_price
            }
          }
        }
        """
        variables = {"handle": handle}
        
        response_data = send_graphql_request(url, query_get_hero_by_handle, variables, token)
        
        if 'errors' in response_data:
            print(f"Error fetching data for handle {handle}: {response_data['errors']}")
            return None
        
        hero_data = response_data.get('data', {}).get('heroes', [])
        if not hero_data:
            print(f"No data found for handle {handle}")
            return None
        
        return hero_data[0]
    
    def process_hero_data(hero_data):
        processed_data = {
            "handle": hero_data["handle"],
            "id": hero_data["id"],
            "inflation_degree": hero_data["distribution_probability"]["inflation_degree"] if hero_data["distribution_probability"] else None
        }
        
        # Process score_history
        score_history = hero_data.get("score_history", [])
        score_history_dict = {}
        closest_to_midnight = {}
    
        for entry in score_history:
            created_at = entry["created_at"]
            date = adjust_date(created_at)
            timestamp = parse_datetime(created_at)
            
            closing_score_key = f"{date} Closing Score"
            closing_rank_key = f"{date} Closing Rank"
            
            # Calculate the difference from midnight
            midnight = datetime.combine(date, datetime.min.time())
            time_diff = abs((timestamp - midnight).total_seconds())
            
            # Update if this timestamp is closer to midnight
            if closing_score_key not in closest_to_midnight or time_diff < closest_to_midnight[closing_score_key]:
                closest_to_midnight[closing_score_key] = time_diff
                score_history_dict[closing_score_key] = entry["fantasy_score"]
                score_history_dict[closing_rank_key] = entry["current_rank"]
    
        processed_data.update(score_history_dict)
        
        # Process tournament_scores
        tournament_scores = hero_data.get("tournament_scores", [])
        for entry in tournament_scores:
            try:
                date = datetime.strptime(entry["created_at"], '%Y-%m-%dT%H:%M:%S.%f').date()
            except ValueError:
                date = datetime.strptime(entry["created_at"], '%Y-%m-%d').date()
            tournament_rank_key = f"{date} Tournament Rank"
            processed_data[tournament_rank_key] = entry["current_rank"]
        
        return processed_data

    # Get data for all heroes and create a DataFrame
    url = "https://fantasy-top.hasura.app/v1/graphql"
    

    
    
    # Add tqdm to the loop for progress tracking
    basic_hero_stats_df = pd.read_csv('data/basic_hero_stats_240802_2203.csv')  # Update this to your actual data loading method
   
    all_hero_data = []
    for handle in tqdm(basic_hero_stats_df["hero_handle"], desc="Processing heroes"):
        hero_data = get_hero_data(handle, url, token)
        if hero_data:
            processed_data = process_hero_data(hero_data)
            all_hero_data.append(processed_data)
        
        # To avoid rate limits, wait between requests
        time.sleep(random.uniform(1, 3))
    
    # Create DataFrame from processed data
    hero_scores = pd.DataFrame(all_hero_data)

    hero_scores = hero_scores.rename(columns={'handle': 'hero_handle', 'id': 'hero_id'})
    
    return hero_scores


### Get hero suppy data - This one takes a while - need to save it to csv and only update periodically

In [95]:
def get_hero_supply():
    query_get_supply_per_hero_id = """
    query GET_SUPPLY_PER_HERO_ID($heroId: String!) @cached(ttl: 3600) {
      rarity1Count: indexer_cards_aggregate(
        where: {hero_id: {_eq: $heroId}, rarity: {_eq: 1}, owner: {_neq: "0x0000000000000000000000000000000000000000"}}
      ) {
        aggregate {
          count
        }
      }
      rarity2Count: indexer_cards_aggregate(
        where: {hero_id: {_eq: $heroId}, rarity: {_eq: 2}, owner: {_neq: "0x0000000000000000000000000000000000000000"}}
      ) {
        aggregate {
          count
        }
      }
      rarity3Count: indexer_cards_aggregate(
        where: {hero_id: {_eq: $heroId}, rarity: {_eq: 3}, owner: {_neq: "0x0000000000000000000000000000000000000000"}}
      ) {
        aggregate {
          count
        }
      }
      rarity4Count: indexer_cards_aggregate(
        where: {hero_id: {_eq: $heroId}, rarity: {_eq: 4}, owner: {_neq: "0x0000000000000000000000000000000000000000"}}
      ) {
        aggregate {
          count
        }
      }
      burnedCardsCount: indexer_cards_aggregate(
        where: {hero_id: {_eq: $heroId}, owner: {_eq: "0x0000000000000000000000000000000000000000"}}
      ) {
        aggregate {
          count
        }
      }
      utilityCount: indexer_cards_aggregate(
        where: {hero_id: {_eq: $heroId}, in_deck: {_eq: true}, owner: {_neq: "0x0000000000000000000000000000000000000000"}}
      ) {
        aggregate {
          count
        }
      }
    }
    """
    
    def process_get_supply_per_hero_id(response, hero_id):
        if 'errors' in response:
            print('Errors:', response['errors'])
            return pd.DataFrame()
        
        data = response.get('data', {})
        
        supply_data = {
            'heroId': hero_id,
            'rarity1Count': data['rarity1Count']['aggregate']['count'],
            'rarity2Count': data['rarity2Count']['aggregate']['count'],
            'rarity3Count': data['rarity3Count']['aggregate']['count'],
            'rarity4Count': data['rarity4Count']['aggregate']['count'],
            'burnedCardsCount': data['burnedCardsCount']['aggregate']['count'],
            'utilityCount': data['utilityCount']['aggregate']['count']
        }
        
        return pd.DataFrame([supply_data])
    
    def get_supply_per_hero_id(url, query, hero_ids, token, delay=1, max_retries=3):
        all_supplies = []
        total_heroes = len(hero_ids)
        
        with tqdm(total=total_heroes, desc="Fetching hero data") as pbar:
            for index, hero_id in enumerate(hero_ids):
                variables = {"heroId": str(hero_id)}  # Ensure heroId is a string
                retries = 0
                while retries < max_retries:
                    try:
                        status_message = f"Fetching data for hero {hero_id} ({index+1}/{total_heroes}), attempt {retries+1}"
                        sys.stdout.write(status_message)
                        sys.stdout.flush()
                        response = send_graphql_request(url, query, variables, token)
                        supply_df = process_get_supply_per_hero_id(response, hero_id)
                        all_supplies.append(supply_df)
                        sys.stdout.write(f"Successfully fetched data for hero {hero_id}          \n")
                        sys.stdout.flush()
                        time.sleep(delay)  # Delay to handle rate limits
                        break
                    except Exception as e:
                        sys.stdout.write(f"Error fetching data for hero {hero_id}: {e}          \r")
                        sys.stdout.flush()
                        retries += 1
                        time.sleep(delay * retries)  # Exponential backoff
                        if retries >= max_retries:
                            sys.stdout.write(f"Failed to fetch data for hero {hero_id} after {max_retries} attempts\n")
                            sys.stdout.flush()
                pbar.update(1)
    
        return pd.concat(all_supplies, ignore_index=True)
    
    hero_ids = hero_stats_df['hero_id'].tolist()
    
    # Get supply data for all heroes
    all_hero_supplies_df = get_supply_per_hero_id(url_graphql, query_get_supply_per_hero_id, hero_ids, token)

    # Step 1: Rename heroId to hero_id in hero_card_supply
    all_hero_supplies_df = all_hero_supplies_df.rename(columns={'heroId': 'hero_id'})
    
    return all_hero_supplies_df


### Get Market Bids - This one takes a while

In [89]:
def get_bids():
    def get_highest_bids_for_hero(hero_id, token, cookies, delay=2, max_retries=3):
        url_rest = "https://fantasy.top/api/bids/get-bid-orders"
        hero_bids = {'hero_id': hero_id}
        
        for rarity in range(1, 5):
            params = {
                'hero_id': hero_id,
                'rarity': rarity,
                'include_orderbook': 'true',
                'include_personal_bids': 'true',
            }
    
            retries = 0
            while retries < max_retries:
                try:
                    status_message = f"Fetching data for hero {hero_id} rarity {rarity}, attempt {retries + 1}\r"
                    sys.stdout.write(status_message)
                    sys.stdout.flush()
                    
                    response = send_graphql_request(url=url_rest, request_type='rest', params=params, token=token, cookies=cookies)
                    
                    # Check if there are bids and get the highest bid
                    highest_bid = 0
                    if response.get('orderbook_bids'):
                        highest_bid = max(int(bid['price']) for bid in response['orderbook_bids'])
                        highest_bid /= 1e18  # Convert to ETH
                    
                    hero_bids[f'rarity{rarity}HighestBid'] = highest_bid
                    break
                except Exception as e:
                    sys.stdout.write(f"Error fetching data for hero {hero_id} rarity {rarity}: {e}, attempt {retries + 1}\r")
                    sys.stdout.flush()
                    retries += 1
                    time.sleep(delay * retries)  # Exponential backoff
            
            if retries >= max_retries:
                sys.stdout.write(f"Failed to fetch data for hero {hero_id} rarity {rarity} after {max_retries} attempts\n")
                sys.stdout.flush()
                hero_bids[f'rarity{rarity}HighestBid'] = None
        
        return hero_bids
    
    def collect_highest_bids(ref_df, token, cookies, delay=7, max_retries=3):
        data = []
        total_heroes = len(ref_df['hero_id'])
        
        for index, hero_id in enumerate(ref_df['hero_id']):
            status_message = f"Processing hero {hero_id} ({index + 1}/{total_heroes})...\r"
            sys.stdout.write(status_message)
            sys.stdout.flush()
            
            hero_bids = get_highest_bids_for_hero(hero_id, token, cookies, delay, max_retries)
            data.append(hero_bids)
            
            # Progress status
            sys.stdout.write(f"Completed {index + 1} out of {total_heroes}\n")
            sys.stdout.flush()
            time.sleep(delay)  # Additional delay to avoid rate limiting
        
        highest_bids_df = pd.DataFrame(data)
        return highest_bids_df
    
    # Example usage
    highest_bids_df = collect_highest_bids(hero_stats_df, token, cookies)

    return highest_bids_df


### Get Last Trade

In [90]:
def get_last_trades():
    # Define the new query
    query_get_last_trade = """
    query GET_LAST_TRADE {
      indexer_trades(
        distinct_on: hero_rarity_index
        order_by: {hero_rarity_index: asc, timestamp: desc}
      ) {
        id
        hero_rarity_index
        price
        timestamp
      }
    }
    """
    
    def process_get_last_trade(response):
        if 'errors' in response:
            print('Errors:', response['errors'])
            return pd.DataFrame()
        
        trades = response.get('data', {}).get('indexer_trades', [])
        
        last_trade_data = []
        for trade in trades:
            hero_id, rarity = trade['hero_rarity_index'].split('_')
            last_trade_data.append({
                'heroId': hero_id,
                'rarity': f'rarity{rarity}',
                'lastSalePrice': convert_to_eth(trade['price']),
                'lastSaleTime': trade['timestamp']
            })
        
        return pd.DataFrame(last_trade_data)
    
    
    def get_last_trade(url, query, token):
        response = send_graphql_request(url, query, token=token)
        last_trade_df = process_get_last_trade(response)
        return last_trade_df
    
    # Make the request and process the data
    last_trade_df = get_last_trade(url_graphql, query_get_last_trade, token)
    
    # Pivot the DataFrame to get the desired format
    pivoted_df = last_trade_df.pivot(index='heroId', columns='rarity', values=['lastSalePrice', 'lastSaleTime'])
    pivoted_df.columns = [f'{col[1]}{col[0]}' for col in pivoted_df.columns]  # Flatten the MultiIndex columns
    pivoted_df.reset_index(inplace=True)

    pivoted_df = pivoted_df.rename(columns={'heroId': 'hero_id'})

    return pivoted_df


## Tournaments

### Get Next Torunament (Submissions)

In [17]:
# # Function to get the next tournaments
# def get_next_tournaments(player_id, token, max_retries=5, delay=2):
#     query_get_next_tournaments = """
#     query GET_NEXT_TOURNAMENTS($playerId: String!, $limit: Int = 10, $offset: Int = 0) {
#       tournaments_tournament(
#         where: {end_date: {_gte: "now()"}, is_visible: {_eq: true}}
#       ) {
#         id
#         name
#         description
#         start_date
#         end_date
#         is_main
#         tournament_number
#         flags
#         league
#         image
#         rewards {
#           type
#           distribution(path: "[0].reward")
#           total_supply
#         }
#         current_players_aggregate(where: {is_registered: {_eq: true}}) {
#           aggregate {
#             count
#           }
#         }
#         is_registered: current_players_aggregate(
#           where: {_and: {player_id: {_eq: $playerId}, is_registered: {_eq: true}}}
#         ) {
#           aggregate {
#             count
#           }
#         }
#       }
#     }
#     """
    
#     variables = {
#         "playerId": player_id,
#     }
    
#     retries = 0
#     while retries < max_retries:
#         response = send_graphql_request(url_graphql, query_get_next_tournaments, variables, token)
#         if 'errors' in response:
#             error_code = response['errors'][0]['extensions']['code']
#             if error_code == 'rate-limit-exceeded':
#                 print(f"Rate limit exceeded, sleeping for {delay} seconds. Attempt {retries + 1}/{max_retries}")
#                 time.sleep(delay)
#                 delay *= 2  # Exponential backoff
#                 retries += 1
#             else:
#                 print(f"Query: {query_get_next_tournaments}")
#                 print(f"Variables: {variables}")
#                 raise Exception(f"Error fetching data: {response['errors']}")
#         else:
#             return response['data']['tournaments_tournament']
#     raise Exception("Max retries exceeded")

# # Example usage to get next tournaments
# player_id = "0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a"


# next_tournaments = get_next_tournaments(player_id, token)
# next_tournaments_df = pd.DataFrame(next_tournaments)


Query: 
    query GET_NEXT_TOURNAMENTS($playerId: String!, $limit: Int = 10, $offset: Int = 0) {
      tournaments_tournament(
        where: {end_date: {_gte: "now()"}, is_visible: {_eq: true}}
      ) {
        id
        name
        description
        start_date
        end_date
        is_main
        tournament_number
        flags
        league
        image
        rewards {
          type
          distribution(path: "[0].reward")
          total_supply
        }
        current_players_aggregate(where: {is_registered: {_eq: true}}) {
          aggregate {
            count
          }
        }
        is_registered: current_players_aggregate(
          where: {_and: {player_id: {_eq: $playerId}, is_registered: {_eq: true}}}
        ) {
          aggregate {
            count
          }
        }
      }
    }
    
Variables: {'playerId': '0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a'}


Exception: Error fetching data: [{'message': 'Could not verify JWT: JWSError (CompactDecodeError Invalid number of parts: Expected 3 parts; got 1)', 'extensions': {'path': '$', 'code': 'invalid-jwt'}}]

### Get Tournament Entries

In [18]:
# # Function to get current tournament player stats
# def get_current_tournament_player_stats(player_id, tournament_id, tournament_id_string, token, max_retries=5, delay=2):
#     query_get_current_tournament_player = """
#     query GET_CURRENT_TOURNAMENT_PLAYER($id: String!, $tournament_id: uuid!, $tournament_id_string: String!) {
#       tournaments_current_players(
#         where: {player_id: {_eq: $id}, tournament_id: {_eq: $tournament_id}}
#       ) {
#         id
#         player_id
#         player {
#           id
#           handle
#           name
#           profile_picture
#           player_trades {
#             rank
#             total_volume
#           }
#           fantasy_points
#           fantasy_points_referrals
#         }
#         score
#         rank
#         previous_rank
#         card1
#         card1_data {
#           id
#           owner
#           gliding_score
#           in_deck
#           picture_url
#           token_id
#           hero_rarity_index
#           sell_order {
#             id
#             price_numeric
#           }
#           hero {
#             id
#             name
#             handle
#             stars
#             profile_image_url_https
#             followers_count
#             current_score {
#               fantasy_score
#               current_rank
#             }
#             tournament_scores(where: {id: {_eq: $tournament_id_string}}) {
#               fantasy_score
#               current_rank
#               views
#             }
#           }
#         }
#         card2
#         card2_data {
#           id
#           owner
#           gliding_score
#           in_deck
#           picture_url
#           token_id
#           hero_rarity_index
#           sell_order {
#             id
#             price_numeric
#           }
#           hero {
#             id
#             name
#             handle
#             stars
#             profile_image_url_https
#             followers_count
#             current_score {
#               fantasy_score
#               current_rank
#             }
#             tournament_scores(where: {id: {_eq: $tournament_id_string}}) {
#               fantasy_score
#               current_rank
#               views
#             }
#           }
#         }
#         card3
#         card3_data {
#           id
#           owner
#           gliding_score
#           in_deck
#           picture_url
#           token_id
#           hero_rarity_index
#           sell_order {
#             id
#             price_numeric
#           }
#           hero {
#             id
#             name
#             handle
#             stars
#             profile_image_url_https
#             followers_count
#             current_score {
#               fantasy_score
#               current_rank
#             }
#             tournament_scores(where: {id: {_eq: $tournament_id_string}}) {
#               fantasy_score
#               current_rank
#               views
#             }
#           }
#         }
#         card4
#         card4_data {
#           id
#           owner
#           gliding_score
#           in_deck
#           picture_url
#           token_id
#           hero_rarity_index
#           sell_order {
#             id
#             price_numeric
#           }
#           hero {
#             id
#             name
#             handle
#             stars
#             profile_image_url_https
#             followers_count
#             current_score {
#               fantasy_score
#               current_rank
#             }
#             tournament_scores(where: {id: {_eq: $tournament_id_string}}) {
#               fantasy_score
#               current_rank
#               views
#             }
#           }
#         }
#         card5
#         card5_data {
#           id
#           owner
#           gliding_score
#           in_deck
#           picture_url
#           token_id
#           hero_rarity_index
#           sell_order {
#             id
#             price_numeric
#           }
#           hero {
#             id
#             name
#             handle
#             stars
#             profile_image_url_https
#             followers_count
#             current_score {
#               fantasy_score
#               current_rank
#             }
#             tournament_scores(where: {id: {_eq: $tournament_id_string}}) {
#               fantasy_score
#               current_rank
#               views
#             }
#           }
#         }
#       }
#     }
#     """
    
#     variables = {
#         "id": player_id,
#         "tournament_id": tournament_id,
#         "tournament_id_string": tournament_id_string
#     }
    
#     retries = 0
#     while retries < max_retries:
#         response = send_graphql_request(url_graphql, query_get_current_tournament_player, variables, token)
#         if 'errors' in response:
#             error_code = response['errors'][0]['extensions']['code']
#             if error_code == 'rate-limit-exceeded':
#                 print(f"Rate limit exceeded, sleeping for {delay} seconds. Attempt {retries + 1}/{max_retries}")
#                 time.sleep(delay)
#                 delay *= 2  # Exponential backoff
#                 retries += 1
#             else:
#                 raise Exception(f"Error fetching data: {response['errors']}")
#         else:
#             return response['data']['tournaments_current_players']
#     raise Exception("Max retries exceeded")

# # Main function to fetch and save tournament data
# def fetch_and_save_tournament_data(player_id, token):
#     try:
#         next_tournaments = get_next_tournaments(player_id, token)
#         next_tournaments_df = pd.DataFrame(next_tournaments)
#         print("Next Tournaments:")
#         print(next_tournaments_df)
        
#         # Iterate through the fetched tournaments and get stats if the player is registered
#         for idx, tournament in next_tournaments_df.iterrows():
#             is_registered = tournament['is_registered']['aggregate']['count']
#             if is_registered > 0:
#                 tournament_id = tournament['id']
#                 simplified_name = tournament['name'].replace(' ', '_')
#                 print(f"Fetching stats for tournament: {simplified_name} (ID: {tournament_id})")
                
#                 try:
#                     tournament_stats = get_current_tournament_player_stats(player_id, tournament_id, tournament_id, token)
                    
#                     # Process the stats to create a DataFrame
#                     card_list = []
#                     entry_id = 1
#                     for entry in tournament_stats:
#                         for i in range(1, 6):
#                             card_key = f'card{i}_data'
#                             if card_key in entry:
#                                 card_data = entry[card_key]
#                                 hero_data = card_data['hero']
#                                 card_info = {
#                                     'entry_id': entry_id,
#                                     'card_id': card_data['id'],
#                                     'card_owner': card_data['owner'],
#                                     'gliding_score': card_data['gliding_score'],
#                                     'card_in_deck': card_data['in_deck'],
#                                     'picture_url': card_data['picture_url'],
#                                     'token_id': card_data['token_id'],
#                                     'hero_rarity_index': card_data['hero_rarity_index'],
#                                     'hero_id': hero_data['id'],
#                                     'hero_name': hero_data['name'],
#                                     'hero_handle': hero_data['handle'],
#                                     'hero_profile_image_url': hero_data['profile_image_url_https'],
#                                     'hero_followers_count': hero_data['followers_count'],
#                                     'hero_stars': hero_data['stars'],
#                                     'hero_fantasy_score': hero_data['current_score']['fantasy_score'],
#                                     'hero_current_rank': hero_data['current_score']['current_rank'],
#                                     'tournament_fantasy_score': hero_data['tournament_scores'][0]['fantasy_score'] if hero_data['tournament_scores'] else 0,
#                                     'tournament_current_rank': hero_data['tournament_scores'][0]['current_rank'] if hero_data['tournament_scores'] else 0,
#                                     'tournament_views': hero_data['tournament_scores'][0]['views'] if hero_data['tournament_scores'] else 0
#                                 }
#                                 card_list.append(card_info)
#                         entry_id += 1
                    
#                     upcoming_tournament_df = pd.DataFrame(card_list)
#                     print(f"DataFrame for {simplified_name} created with {len(upcoming_tournament_df)} rows")
#                     globals()[f"upcoming_{simplified_name}_df"] = upcoming_tournament_df  # Save DataFrame with the appropriate name
#                     save_df_as_csv(upcoming_tournament_df, f"upcoming_{simplified_name}", folder="data")
#                 except Exception as e:
#                     print(f"Failed to fetch data for tournament {tournament_id} with error: {str(e)}")
#     except Exception as e:
#         print(str(e))

### Get Tournaments List

In [20]:
# Define the new query
query_get_tournaments_by_time = """
query GET_TOURNAMENTS_BY_TIME($gte: timestamptz!, $lte: timestamptz!, $player_id: String!) {
  tournaments_tournament(
    where: {start_date: {_gte: $gte, _lte: $lte}}
    order_by: {start_date: desc}
  ) {
    id
    name
    description
    start_date
    end_date
    is_main
    league
    tournament_number
    player_history_count: players_history_aggregate(
      where: {player_id: {_eq: $player_id}}
    ) {
      aggregate {
        count
      }
    }
    total_players_count: players_history_aggregate {
      aggregate {
        count
      }
    }
    rewards {
      type
      distribution(path: "[0].reward")
      total_supply
    }
  }
}
"""

def parse_datetime(date_str):
    try:
        return datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S.%fZ')
    except ValueError:
        return datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S%z')
        

def process_get_tournaments_by_time(response):
    if 'errors' in response:
        print('Errors:', response['errors'])
        return pd.DataFrame()
    
    tournaments = response.get('data', {}).get('tournaments_tournament', [])
    
    tournaments_data = []
    for tournament in tournaments:
        rewards_data = tournament.get('rewards', [{}])[0]
        tournaments_data.append({
            'id': tournament['id'],
            'name': tournament['name'],
            'description': tournament['description'],
            'start_date': tournament['start_date'],
            'end_date': tournament['end_date'],
            'start_day': parse_datetime(tournament['start_date']).strftime('%A'), 
            'end_day': parse_datetime(tournament['end_date']).strftime('%A'),
            'is_main': tournament['is_main'],
            'league': tournament['league'],
            'tournament_number': tournament['tournament_number'],
            'player_history_count': tournament['player_history_count']['aggregate']['count'],
            'total_players_count': tournament['total_players_count']['aggregate']['count'],
            'reward_type': rewards_data.get('type'),
            'reward_distribution': rewards_data.get('distribution'),
            'reward_total_supply': rewards_data.get('total_supply')
        })
    
    return pd.DataFrame(tournaments_data)

def get_tournaments_by_time(url, query, gte, lte, player_id, token):
    variables = {
        "gte": gte,
        "lte": lte,
        "player_id": player_id
    }
    
    response = send_graphql_request(url, query, variables, token=token)
    tournaments_df = process_get_tournaments_by_time(response)
    return tournaments_df

# Example usage
gte = "1970-01-01T00:00:00.000Z"
lte = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
player_id = ""  

tournaments_df = get_tournaments_by_time(url_graphql, query_get_tournaments_by_time, gte, lte, player_id, token)

# Display the DataFrame
print(tournaments_df)

                                      id                         name  \
0   d257197f-15e3-440f-bd02-3e3546486d46                         Gold   
1   1190639b-58b5-407f-b6d0-accaebf3ee05                       Bronze   
2   11413b0b-3798-4e54-8d3a-c059588cdd6c                        Elite   
3   816ae5aa-2cdb-4631-8719-1f03c648de8e                       Silver   
4   a7863482-05c1-43d3-af29-54107a9fbdaf                        Elite   
5   563eb931-fd4e-4f53-8c44-4e7339da825b                         Gold   
6   9c9bd433-a346-4ea9-b845-0fde7fdac5c9                       Silver   
7   6d37c0f0-671c-44df-9fbd-ccd4fb827d35                       Bronze   
8   96a55828-f71a-4c3f-b5f2-0eb883f97f6a                        Elite   
9   a0fd9f04-4c53-4c78-8984-8ffc1e1f9f0c                       Silver   
10  7f1459b5-9935-4eef-a26f-ae9ce3224cb8                       Bronze   
11  d0cb589a-2ddb-40d1-880c-37aa1f021774                         Gold   
12  453e8ff2-26c8-4b1b-8d4c-7ea102bd5953           

In [21]:
tournaments_df

,id,name,description,start_date,end_date,start_day,end_day,is_main,league,tournament_number,player_history_count,total_players_count,reward_type,reward_distribution,reward_total_supply
0,d257197f-15e3-440f-bd02-3e3546486d46,Gold,Gold main 12,2024-08-12T15:00:00+00:00,2024-08-15T14:00:00+00:00,Monday,Thursday,True,2.0,12.0,0,2585,STAR,NaN,1
1,1190639b-58b5-407f-b6d0-accaebf3ee05,Bronze,Bronze main 12,2024-08-12T15:00:00+00:00,2024-08-15T14:00:00+00:00,Monday,Thursday,True,4.0,12.0,0,9463,GOLD,260.0,40000
2,11413b0b-3798-4e54-8d3a-c059588cdd6c,Elite,Elite main 12,2024-08-12T15:00:00+00:00,2024-08-15T14:00:00+00:00,Monday,Thursday,True,1.0,12.0,0,1531,STAR,NaN,1
3,816ae5aa-2cdb-4631-8719-1f03c648de8e,Silver,Silver main 12,2024-08-12T15:00:00+00:00,2024-08-15T14:00:00+00:00,Monday,Thursday,True,3.0,12.0,0,4613,STAR,NaN,1
4,a7863482-05c1-43d3-af29-54107a9fbdaf,Elite,Elite main 11,2024-08-05T15:00:00+00:00,2024-08-08T14:00:00+00:00,Monday,Thursday,True,1.0,11.0,0,1532,STAR,NaN,1
5,563eb931-fd4e-4f53-8c44-4e7339da825b,Gold,Gold main 11,2024-08-05T15:00:00+00:00,2024-08-08T14:00:00+00:00,Monday,Thursday,True,2.0,11.0,0,2729,STAR,NaN,1
6,9c9bd433-a346-4ea9-b845-0fde7fdac5c9,Silver,Silver main 11,2024-08-05T15:00:00+00:00,2024-08-08T14:00:00+00:00,Monday,Thursday,True,3.0,11.0,0,4422,STAR,NaN,1
7,6d37c0f0-671c-44df-9fbd-ccd4fb827d35,Bronze,Bronze main 11,2024-08-05T15:00:00+00:00,2024-08-08T14:00:00+00:00,Monday,Thursday,True,4.0,11.0,0,8737,STAR,NaN,1
8,96a55828-f71a-4c3f-b5f2-0eb883f97f6a,Elite,Elite main 10,2024-07-29T15:00:00+00:00,2024-08-01T14:00:00+00:00,Monday,Thursday,True,1.0,10.0,0,1579,STAR,NaN,1
9,a0fd9f04-4c53-4c78-8984-8ffc1e1f9f0c,Silver,Silver main 10,2024-07-29T15:00:00+00:00,2024-08-01T14:00:00+00:00,Monday,Thursday,True,3.0,10.0,0,4459,STAR,NaN,1


In [20]:
# Assume your DataFrame is named tournaments_df

# Drop duplicate rows where both start_date and end_date are the same
simplified_tournaments_df = tournaments_df.drop_duplicates(subset=['start_date', 'end_date'])

simplified_tournaments_df


""


In [ ]:
import re

# First, sort the DataFrame by start_date to ensure correct numbering for "Main" tournaments
simplified_tournaments_df = simplified_tournaments_df.sort_values(by='start_date')

# Function to cleanse the simplified name
def cleanse_name(name):
    name = re.sub(r'[^\w\s]', '', name)  # Remove all non-alphanumeric characters except whitespace
    name = re.sub(r'\s+', ' ', name)  # Replace multiple spaces with a single space
    return name.strip()  # Remove leading and trailing spaces

# Create a column for the simplified names
simplified_names = []

# Initialize the main tournament counter
main_counter = 1

# Iterate through the DataFrame to set the simplified names
for idx, row in simplified_tournaments_df.iterrows():
    if row['is_main']:
        simplified_name = f"Main {main_counter}"
        main_counter += 1
    else:
        simplified_name = cleanse_name(row['name'])  # Cleanse the name to remove special characters
    
    simplified_names.append(simplified_name)

# Add the new column to the DataFrame
simplified_tournaments_df['simplified_name'] = simplified_names

# Display the updated DataFrame
print(simplified_tournaments_df)


In [31]:
tournament_scores = pd.read_csv('data/tournament_scores_240811_1115.csv')


### Get Tournament Stats - API - Doesn't appear to work for all tournaments - DO NOT USE - Use CSV copied from https://docs.google.com/spreadsheets/d/1YkWu1O80VeGnUwO1CZ1tGfjTHxEyRhOn-Li08-wEd6I/edit?gid=0#gid=0

In [23]:
# Function to get tournament stats for a specific tournament_id
def get_tournament_stats(tournament_id, token=token):
    query_get_heros_with_stats_tournament = """
    query GET_HEROS_WITH_STATS_TOURNAMENT($tournament_id: String = "", $offset: Int = 0, $limit: Int = 20, $order_by: [twitter_data_tournament_history_order_by!] = {current_rank: asc}, $search: String = "") {
      twitter_data_current: twitter_data_tournament_history(
        where: {id: {_eq: $tournament_id}, hero: {_or: [{name: {_ilike: $search}}, {handle: {_ilike: $search}}]}}
        order_by: $order_by
        offset: $offset
        limit: $limit
      ) {
        current_rank
        previous_rank
        views
        tweet_count
        fantasy_score
        reach
        avg_views
        hero {
          followers_count
          name
          handle
          profile_image_url_https
          volume: trades_aggregate {
            aggregate {
              sum {
                price
              }
            }
          }
          last_sale: trades(limit: 1) {
            price
          }
          floor: unique_sell_orders(order_by: {lowest_price: asc_nulls_last}, limit: 1) {
            lowest_price
          }
        }
      }
    }
    """
    
    def process_get_heros_with_stats_tournament(response):
        if 'errors' in response:
            print('Errors:', response['errors'])
            return pd.DataFrame()
        
        heros = response.get('data', {}).get('twitter_data_current', [])
        
        heros_data = []
        for hero_entry in heros:
            hero = hero_entry['hero']
            heros_data.append({
                'current_rank': hero_entry['current_rank'],
                'previous_rank': hero_entry['previous_rank'],
                'views': hero_entry['views'],
                'tweet_count': hero_entry['tweet_count'],
                'fantasy_score': hero_entry['fantasy_score'],
                'reach': hero_entry['reach'],
                'avg_views': hero_entry['avg_views'],
                'hero_followers_count': hero['followers_count'],
                'hero_name': hero['name'],
                'hero_handle': hero['handle'],
                'hero_profile_image_url': hero['profile_image_url_https'],
                'hero_volume': convert_to_eth(hero['volume']['aggregate']['sum']['price']),
                'hero_last_sale_price': convert_to_eth(hero['last_sale'][0]['price']),
                'hero_floor_price': convert_to_eth(hero['floor'][0]['lowest_price'])
            })
        
        return pd.DataFrame(heros_data)
    
    def get_heros_with_stats_tournament(url, query, tournament_id, offset, limit, order_by, search, token, max_retries=5, delay=2):
        variables = {
            "tournament_id": tournament_id,
            "offset": offset,
            "order_by": order_by,
            "limit": limit,
            "search": search
        }
        
        retries = 0
        while retries < max_retries:
            response = send_graphql_request(url, query, variables, token=token)
            if 'errors' in response:
                error_code = response['errors'][0]['extensions']['code']
                if error_code == 'rate-limit-exceeded':
                    print(f"Rate limit exceeded, sleeping for {delay} seconds. Attempt {retries + 1}/{max_retries}")
                    time.sleep(delay)
                    delay *= 2  # Exponential backoff
                    retries += 1
                else:
                    print(f"Query: {query}")
                    print(f"Variables: {variables}")
                    raise Exception(f"Error fetching data: {response['errors']}")
            else:
                return process_get_heros_with_stats_tournament(response)
        raise Exception("Max retries exceeded")
    
    # Get hero stats for the specified tournament
    offset = 0
    order_by = {"fantasy_score": "desc"}
    limit = 200
    search = "%%"

    
    heros_with_stats_tournament_df = get_heros_with_stats_tournament(url_graphql, query_get_heros_with_stats_tournament, tournament_id, offset, limit, order_by, search, token)
    
    return heros_with_stats_tournament_df




In [26]:
last_tournament = get_tournament_stats("11413b0b-3798-4e54-8d3a-c059588cdd6c", token=token)

In [32]:
tournament_scores

,Name,Handle,Main 11,Main 10,Main 9,Main 8,Main 7,Main 6 *Sat/Sun Only*,Main 5,All Rarities | 22 days,Main 4,Main 3,Common Only ✳️ Capped 15 🌟,Rare Only 💠,Common Only ✳️ Capped 20 🌟,Main 2,Main 1,Flash Tournament
0,greg,greg16676935420,996,990,999,999,994,945,995,955,949,862,900,941,"1,000","1,000",757,768
1,wallstreetbets,wallstreetbets,986,902,878,899,819,916,913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GBH Remilio,xbtGBH,868,960,933,894,945,899,908,986,930,992,"1,000",951,906,951,"1,000","1,000"
3,Ansem,blknoiz06,954,931,930,888,910,956,956,986,958,"1,000",970,929,940,971,945,889
4,Evanss6,Evan_ss6,943,888,887,842,858,936,855,861,824,921,903,906,771,842,581,337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,Pentoshi euroPeng,Pentosh1,0,583,0,764,0,778,642,689,713,771,482,613,456,502,NaN,NaN
173,loomdart Holy War Arc,loomdart,0,315,76,171,151,429,454,534,609,478,272,214,0,64,479,547
174,Bharat Krymo,krybharat,0,0,141,124,60,15,22,52,113,81,59,144,69,15,72,124
175,wab eth,wabdoteth,0,0,250,0,0,517,502,365,460,382,344,189,460,418,NaN,NaN


In [30]:
last_tournament.tail(20)

,current_rank,previous_rank,views,tweet_count,fantasy_score,reach,avg_views,hero_followers_count,hero_name,hero_handle,hero_profile_image_url,hero_volume,hero_last_sale_price,hero_floor_price
157,158,158,1492,2,49.915178,7.048375,746.0,10527,gleb,glebble,https://pbs.twimg.com/profile_images/179318049...,202.348566,0.020000,0.0
158,159,159,1355,1,38.354717,5.572004,1355.0,27707,Sandra,sandraaleow,https://pbs.twimg.com/profile_images/177697825...,41.932662,0.014500,0.0
159,160,160,2166,3,25.670548,2.452446,722.0,36229,palmer,bitcoinPalmer,https://pbs.twimg.com/profile_images/182267808...,280.506011,0.010000,0.0
160,161,161,498,1,8.942191,1.406124,498.0,36770,Levi 💎⚡️,0xCaptainLevi,https://pbs.twimg.com/profile_images/167045639...,38.035304,0.100000,NaN
161,162,162,0,0,0.000000,0.000000,0.0,32469,Unipcs (aka 'Bonk Guy') 🎒,theunipcs,https://pbs.twimg.com/profile_images/175589988...,12.110626,0.015747,0.0
162,162,162,0,0,0.000000,0.000000,0.0,41618,BIG DICK BULL (BDB),BigDickBull69,https://pbs.twimg.com/profile_images/179335102...,145.737769,0.200000,0.0
163,162,162,0,0,0.000000,0.000000,0.0,12196,WarDaddyCapital,wardaddycapital,https://pbs.twimg.com/profile_images/152691849...,618.154455,0.015000,NaN
164,162,162,0,0,0.000000,0.000000,0.0,52902,blockgraze,blockgraze,https://pbs.twimg.com/profile_images/176065556...,130.359192,1.000000,0.0
165,162,162,0,0,0.000000,0.000000,0.0,34323,Tom Schmidt ＞|＜,tomhschmidt,https://pbs.twimg.com/profile_images/177372148...,273.862454,0.010000,0.0
166,162,162,0,0,0.000000,0.000000,0.0,36358,WR☻NGUSER ✗,wronguser000,https://pbs.twimg.com/profile_images/161136475...,30.833502,0.115000,0.0


In [43]:
updated_tournament_scores = tournament_scores.merge(last_tournament[['hero_handle', 'fantasy_score']], left_on='Handle', right_on='hero_handle')
updated_tournament_scores.drop(columns=['hero_handle'])
updated_tournament_scores.rename(columns={'fantasy_score': 'Main 12'}, inplace=True)
# Step 1: Get the list of columns
columns = updated_tournament_scores.columns.tolist()

# Step 2: Remove 'Main 12' from its current position
columns.remove('Main 12')

# Step 3: Find the position of 'Main 11'
main_11_index = columns.index('Main 11')

# Step 4: Insert 'Main 12' before 'Main 11'
columns.insert(main_11_index, 'Main 12')

# Step 5: Reorder the DataFrame using the new columns list
updated_tournament_scores = updated_tournament_scores[columns]

updated_tournament_scores['Main 12'] = updated_tournament_scores['Main 12'].round(0).astype(int)


In [45]:
updated_tournament_scores.to_csv('data/tournament_scores_240816_0327.csv')

In [77]:
def get_all_tournaments_stats(simplified_tournaments_df, token):
    for idx, row in simplified_tournaments_df.iterrows():
        tournament_id = row['id']
        simplified_name = row['simplified_name'].replace(' ', '_')
        
        print(f"Fetching stats for tournament: {simplified_name} (ID: {tournament_id})")
        
        try:
            tournament_stats_df = get_tournament_stats(tournament_id, token)
            globals()[simplified_name] = tournament_stats_df
            print(f"DataFrame for {simplified_name} created with {len(tournament_stats_df)} rows")
        except Exception as e:
            print(f"Failed to fetch data for tournament {tournament_id} with error: {str(e)}")

# Example usage:
# Assume you have a DataFrame 'simplified_tournaments_df' and a valid 'token'
get_all_tournaments_stats(simplified_tournaments_df, token)

Fetching stats for tournament: Flash_Tournament (ID: 50d1bb18-c18e-4502-b9cd-e0bc9fd709c6)
DataFrame for Flash_Tournament created with 123 rows
Fetching stats for tournament: Main_1 (ID: ad0cb5e8-0a8a-40a8-9b7d-07dc4441f54b)
DataFrame for Main_1 created with 123 rows
Fetching stats for tournament: Main_2 (ID: d292083b-3516-4754-8fd6-f7417472290c)
DataFrame for Main_2 created with 0 rows
Fetching stats for tournament: Common_Only_Capped_20 (ID: fd1df283-de8b-41bf-b06f-d5a6160d8cf2)
DataFrame for Common_Only_Capped_20 created with 166 rows
Fetching stats for tournament: Rare_Only (ID: 8b10c080-34af-4005-b66e-8afef943566f)
DataFrame for Rare_Only created with 166 rows
Fetching stats for tournament: Common_Only_Capped_15 (ID: 5cca59e3-c861-439e-84da-8e3e6ff0842a)
DataFrame for Common_Only_Capped_15 created with 166 rows
Fetching stats for tournament: All_Rarities_22_days (ID: 941cb41d-6cb9-4fc1-8f4d-1acd843ce219)
DataFrame for All_Rarities_22_days created with 166 rows
Fetching stats for t

In [81]:
test_df = get_tournament_stats('cd041022-d3fd-4966-964f-2cbf3d13645f', token=token)

# Run Functions To Gather Data

In [91]:
import glob

def get_latest_hero_stats_file(directory, prefix='hero_stats'):
    # Use glob to find all files that match the pattern
    files = glob.glob(os.path.join(directory, f'{prefix}*.csv'))
    
    # If no files found, raise an error or handle accordingly
    if not files:
        raise FileNotFoundError(f"No files starting with '{prefix}' found in {directory}")

    # Sort files by modification time, newest first
    latest_file = max(files, key=os.path.getmtime)
    
    return latest_file

# Use the function to get the latest hero_stats file
latest_hero_stats_file = get_latest_hero_stats_file('data')

# Load the latest hero_stats file into a DataFrame
hero_stats_df = pd.read_csv(latest_hero_stats_file)




In [96]:

# print_runtime(lambda: save_df_as_csv(get_hero_stats(), 'hero_stats'))  # This one takes 7 mins
# print_runtime(lambda: save_df_as_csv(download_portfolio(), 'portfolio'))
# print_runtime(lambda: save_df_as_csv(download_basic_hero_stats(), 'basic_hero_stats'))
# print_runtime(lambda: save_df_as_csv(download_listings(), 'listings'))
# print_runtime(lambda: save_df_as_csv(get_last_trades(), 'last_trades'))
print_runtime(lambda: save_df_as_csv(get_hero_supply(), 'hero_card_supply'))
print_runtime(lambda: save_df_as_csv(get_bids(), 'bids'))




Calling <lambda>


Fetching hero data:   0%|                                                                      | 0/177 [00:00<?, ?it/s]

Fetching data for hero 1356434353623093249 (1/177), attempt 1Successfully fetched data for hero 1356434353623093249          


Fetching hero data:   1%|▎                                                             | 1/177 [00:01<03:28,  1.19s/it]

Fetching data for hero 262018615 (2/177), attempt 1Successfully fetched data for hero 262018615          


...

Fetching hero data:  99%|███████████████████████████████████████████████████████████▋| 176/177 [03:34<00:01,  1.22s/it]

Fetching data for hero 1372325396461580292 (177/177), attempt 1Successfully fetched data for hero 1372325396461580292          


Fetching hero data: 100%|████████████████████████████████████████████████████████████| 177/177 [03:35<00:00,  1.22s/it]

DataFrame saved as data\hero_card_supply_240811_0733.csv
<lambda> took 215.7354 seconds to execute
Calling <lambda>


Completed 1 out of 177 1356434353623093249 rarity 4, attempt 1
Completed 2 out of 177 262018615 rarity 4, attempt 1
Completed 3 out of 177 982070158125682688 rarity 4, attempt 1
Failed to fetch data for hero 973261472 rarity 4 after 3 attempts: Too Many Requests for url: https://fantasy.top/api/bids/get-bid-orders?hero_id=973261472&rarity=4&include_orderbook=true&include_personal_bids=true, attempt 3
Completed 4 out of 177
Completed 5 out of 177 1441835930889818113 rarity 4, attempt 1
Completed 6 out of 177 906234475604037637 rarity 4, attempt 1
Completed 7 out of 177 588569122 rarity 4, attempt 1
Failed to fetch data for hero 965651 rarity 4 after 3 attempts: Too Many Requests for url: https://fantasy.top/api/bids/get-bid-orders?hero_id=965651&rarity=4&include_orderbook=true&include_personal_bids=true, attempt 3
Completed 8 out of 177
Completed 9 out of 177 279828819 rarity 4, attempt 1
Completed 10 out of 1771466010689726783491 rarity 4, attempt 1
Completed 11 out of 1771446541960181